## **Federated Learning for attack classification: 5 nodes sharing gradients**

IDs from this file = **idcat5xy** (x = 0 if experiment with dataset, x = 1 if epochs & iterations, y being integer equal or greater than 0)

In this file, experiments with different datasets, and number of epochs & iterations are done. The experiments are divided into sections, based on the elements being changed. 

### Static elements for all experiments (execute first)

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Preprocessing for test and training datasets (to be able to train and make predictions to evaluate)
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Creating categories dataframe
    data_labels = pd.DataFrame()

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)
        
    # Transform to One Hot Encoding the Categories - normal, dos, reconnaissance, generic, exploits, worms, fuzzers, analysis, backdoor, shellcode
    data_labels.insert(0, 'dos', data['attack_cat'].replace('dos', 1).replace(['reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(1, 'reconnaissance', data['attack_cat'].replace('reconnaissance', 1).replace([ 'dos', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(2, 'generic', data['attack_cat'].replace('generic', 1).replace(['dos', 'reconnaissance', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(3, 'exploits', data['attack_cat'].replace('exploits', 1).replace([ 'dos', 'reconnaissance', 'generic', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Generate 2 new columns to fit with training
    auxCol=data_features['sbytes']
    auxCol=0

      # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(Dense(108, input_shape=input_shape, activation='relu'))
    model.add(layers.Conv2D(filters=16, kernel_size=(1, 2), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(1, 2), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(108, activation='relu'))
    model.add(Dense(4, activation='softmax'))

    return model 

In [4]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, labels = [i for i in range(4)], target_names=['dos', 'reconnaissance', 'generic', 'exploits'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

### Experiments with datasets

### Cat5A

In [5]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5A-Part5.csv')
test_cat = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Categories.csv')
test_cat_eq = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Categories_Eq.csv')

In [6]:

num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [7]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'IDCAT500.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [8]:
def weiszfeld_update(points, tol=1e-5, max_iter=100):
    """
    Compute the geometric median using Weiszfeld's algorithm.

    Args:
    points: A list of points (numpy arrays) to find the geometric median of.
    tol: Tolerance for stopping criterion.
    max_iter: Maximum number of iterations.

    Returns:
    The geometric median of the points.
    """
    points = np.array(points)
    median = np.mean(points, axis=0)

    for _ in range(max_iter):
        distances = np.linalg.norm(points - median, axis=1)
        nonzero_distances = np.where(distances != 0, distances, np.finfo(float).eps)
        weights = 1 / nonzero_distances
        new_median = np.sum(points * weights[:, None], axis=0) / np.sum(weights)

        if np.linalg.norm(new_median - median) < tol:
            return new_median

        median = new_median

    return median

def aggregate(grad_list):
    """
    Apply Geometric Median aggregation method to a list of gradients.

    Args:
    grad_list: List of gradients from different models. Each element in the list
               is a list of gradients for each layer of a model.

    Returns:
    The geometric median of the gradients.
    """
    # Flatten gradients to compute geometric median
    flat_grads = [tf.concat([tf.reshape(g, [-1]) for g in grad], axis=0).numpy() for grad in grad_list]

    # Compute geometric median using Weiszfeld's algorithm
    flat_median = weiszfeld_update(flat_grads)

    # Reshape the flat median back to the original shape
    median_grad = []
    index = 0
    for grad in grad_list[0]:
        shape = tf.shape(grad)
        size = tf.reduce_prod(shape)
        median_grad.append(tf.reshape(flat_median[index:index + size], shape))
        index += size

    return median_grad

In [9]:
global_model = build_model((24,1,1))

In [10]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)
xc, yc = preprocessing(test_cat)
xce, yce = preprocessing(test_cat_eq)

In [11]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [12]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3:
            x, y = x4, y4
        elif node == 4:
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xc, yc) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xce, yce)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idCAT500.hdf5')

Epoch 1/5
31/31 [==============================] - 6s 182ms/step - loss: 0.6941 - accuracy: 0.8972 - val_loss: 1.4645 - val_accuracy: 0.1511
Epoch 2/5
31/31 [==============================] - 3s 102ms/step - loss: 0.1227 - accuracy: 0.9986 - val_loss: 1.4735 - val_accuracy: 0.1450
Epoch 3/5
31/31 [==============================] - 3s 103ms/step - loss: 0.0264 - accuracy: 0.9996 - val_loss: 1.5045 - val_accuracy: 0.1433
Epoch 4/5
31/31 [==============================] - 5s 150ms/step - loss: 0.0114 - accuracy: 0.9999 - val_loss: 1.5433 - val_accuracy: 0.1426
Epoch 5/5
31/31 [==============================] - 4s 143ms/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 1.5912 - val_accuracy: 0.1423
Epoch 1/5
31/31 [==============================] - 6s 179ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 1.7772 - val_accuracy: 0.1415
Epoch 2/5
31/31 [==============================] - 4s 114ms/step - loss: 4.1743e-04 - accuracy: 1.0000 - val_loss: 2.0060 - val_accuracy: 0.1411
Epoch 3/5

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


119/119 [==============================] - 0s 4ms/step - loss: 10.4740 - accuracy: 0.2500


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
31/31 [==============================] - 5s 174ms/step - loss: 5.3607e-06 - accuracy: 1.0000 - val_loss: 15.1904 - val_accuracy: 0.1411
Epoch 2/5
31/31 [==============================] - 3s 98ms/step - loss: 2.4675e-07 - accuracy: 1.0000 - val_loss: 15.7732 - val_accuracy: 0.1411
Epoch 3/5
31/31 [==============================] - 4s 116ms/step - loss: 1.2261e-07 - accuracy: 1.0000 - val_loss: 15.8485 - val_accuracy: 0.1411
Epoch 4/5
31/31 [==============================] - 4s 129ms/step - loss: 1.2423e-07 - accuracy: 1.0000 - val_loss: 16.1327 - val_accuracy: 0.1411
Epoch 5/5
31/31 [==============================] - 4s 119ms/step - loss: 1.2315e-07 - accuracy: 1.0000 - val_loss: 16.2319 - val_accuracy: 0.1411
Epoch 1/5
31/31 [==============================] - 4s 134ms/step - loss: 9.1565e-08 - accuracy: 1.0000 - val_loss: 16.3337 - val_accuracy: 0.1411
Epoch 2/5
31/31 [==============================] - 4s 118ms/step - loss: 9.1498e-09 - accuracy: 1.0000 - val_loss: 16.4181 - 

...

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
31/31 [==============================] - 4s 119ms/step - loss: 4.3327e-08 - accuracy: 1.0000 - val_loss: 17.0160 - val_accuracy: 0.1411
Epoch 2/5
31/31 [==============================] - 4s 142ms/step - loss: 3.8384e-08 - accuracy: 1.0000 - val_loss: 16.9384 - val_accuracy: 0.1411
Epoch 3/5
31/31 [==============================] - 4s 127ms/step - loss: 6.9814e-08 - accuracy: 1.0000 - val_loss: 17.0983 - val_accuracy: 0.1411
Epoch 4/5
31/31 [==============================] - 3s 93ms/step - loss: 3.7049e-07 - accuracy: 1.0000 - val_loss: 16.5081 - val_accuracy: 0.1411
Epoch 5/5
31/31 [==============================] - 3s 105ms/step - loss: 3.3877e-08 - accuracy: 1.0000 - val_loss: 17.0286 - val_accuracy: 0.1411
Epoch 1/5
31/31 [==============================] - 5s 156ms/step - loss: 1.5385e-09 - accuracy: 1.0000 - val_loss: 17.1206 - val_accuracy: 0.1411
Epoch 2/5
31/31 [==============================] - 6s 188ms/step - loss: 9.8708e-10 - accuracy: 1.0000 - val_loss: 17.2557 - 

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


119/119 [==============================] - 1s 4ms/step - loss: 12.3380 - accuracy: 0.2500


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[3.3721423149108887, 10.473995208740234], [3.7639644145965576, 11.68632698059082], [3.898350238800049, 12.337974548339844]]
Accuracy for iterations:  [[0.8319951891899109, 0.25], [0.8319796919822693, 0.25], [0.8319796919822693, 0.25]]
F1 for iterations:  [[0.7557386333142229, 0.1], [0.7557013693602648, 0.1], [0.7557013693602648, 0.1]]
Precision for iterations:  [[0.6959233586435923, 0.0625], [0.6922352459557739, 0.0625], [0.6922352459557739, 0.0625]]
Recall for iterations:  [[0.831995163990328, 0.25], [0.8319796639593279, 0.25], [0.8319796639593279, 0.25]]


### Cat5B

In [14]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5B-Part5.csv')
test_cat = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Categories.csv')
test_cat_eq = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Categories_Eq.csv')

In [15]:

num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [16]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'IDCAT501.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [17]:
def weiszfeld_update(points, tol=1e-5, max_iter=100):
    """
    Compute the geometric median using Weiszfeld's algorithm.

    Args:
    points: A list of points (numpy arrays) to find the geometric median of.
    tol: Tolerance for stopping criterion.
    max_iter: Maximum number of iterations.

    Returns:
    The geometric median of the points.
    """
    points = np.array(points)
    median = np.mean(points, axis=0)

    for _ in range(max_iter):
        distances = np.linalg.norm(points - median, axis=1)
        nonzero_distances = np.where(distances != 0, distances, np.finfo(float).eps)
        weights = 1 / nonzero_distances
        new_median = np.sum(points * weights[:, None], axis=0) / np.sum(weights)

        if np.linalg.norm(new_median - median) < tol:
            return new_median

        median = new_median

    return median

def aggregate(grad_list):
    """
    Apply Geometric Median aggregation method to a list of gradients.

    Args:
    grad_list: List of gradients from different models. Each element in the list
               is a list of gradients for each layer of a model.

    Returns:
    The geometric median of the gradients.
    """
    # Flatten gradients to compute geometric median
    flat_grads = [tf.concat([tf.reshape(g, [-1]) for g in grad], axis=0).numpy() for grad in grad_list]

    # Compute geometric median using Weiszfeld's algorithm
    flat_median = weiszfeld_update(flat_grads)

    # Reshape the flat median back to the original shape
    median_grad = []
    index = 0
    for grad in grad_list[0]:
        shape = tf.shape(grad)
        size = tf.reduce_prod(shape)
        median_grad.append(tf.reshape(flat_median[index:index + size], shape))
        index += size

    return median_grad

In [18]:
global_model = build_model((24,1,1))

In [19]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)
xc, yc = preprocessing(test_cat)
xce, yce = preprocessing(test_cat_eq)

In [20]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [21]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3:
            x, y = x4, y4
        elif node == 4:
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xc, yc) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xce, yce)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idCAT501.hdf5')

Epoch 1/5
2/2 [==============================] - 3s 1s/step - loss: 1.4472 - accuracy: 0.2849 - val_loss: 1.4165 - val_accuracy: 0.0000e+00
Epoch 2/5
2/2 [==============================] - 0s 95ms/step - loss: 1.3852 - accuracy: 0.2991 - val_loss: 1.4208 - val_accuracy: 0.0000e+00
Epoch 3/5
2/2 [==============================] - 0s 80ms/step - loss: 1.3282 - accuracy: 0.3227 - val_loss: 1.4250 - val_accuracy: 0.0000e+00
Epoch 4/5
2/2 [==============================] - 0s 86ms/step - loss: 1.2771 - accuracy: 0.3463 - val_loss: 1.4294 - val_accuracy: 0.0000e+00
Epoch 5/5
2/2 [==============================] - 0s 86ms/step - loss: 1.2278 - accuracy: 0.3877 - val_loss: 1.4338 - val_accuracy: 0.0000e+00
Epoch 1/5
2/2 [==============================] - 4s 2s/step - loss: 1.1674 - accuracy: 0.5278 - val_loss: 1.4377 - val_accuracy: 0.0000e+00
Epoch 2/5
2/2 [==============================] - 1s 356ms/step - loss: 1.1246 - accuracy: 0.5892 - val_loss: 1.4419 - val_accuracy: 0.0000e+00
Epoch 3/5

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


119/119 [==============================] - 1s 12ms/step - loss: 1.3251 - accuracy: 0.5066


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
2/2 [==============================] - 1s 694ms/step - loss: 0.7867 - accuracy: 0.6469 - val_loss: 1.4989 - val_accuracy: 0.0000e+00
Epoch 2/5
2/2 [==============================] - 0s 174ms/step - loss: 0.7821 - accuracy: 0.6542 - val_loss: 1.4973 - val_accuracy: 0.0000e+00
Epoch 3/5
2/2 [==============================] - 0s 142ms/step - loss: 0.7760 - accuracy: 0.6563 - val_loss: 1.4955 - val_accuracy: 0.0000e+00
Epoch 4/5
2/2 [==============================] - 1s 252ms/step - loss: 0.7705 - accuracy: 0.6584 - val_loss: 1.4937 - val_accuracy: 0.0000e+00
Epoch 5/5
2/2 [==============================] - 1s 325ms/step - loss: 0.7666 - accuracy: 0.6626 - val_loss: 1.4924 - val_accuracy: 0.0000e+00
Epoch 1/5
2/2 [==============================] - 4s 2s/step - loss: 0.7747 - accuracy: 0.6375 - val_loss: 1.4900 - val_accuracy: 0.0000e+00
Epoch 2/5
2/2 [==============================] - 1s 274ms/step - loss: 0.7691 - accuracy: 0.6443 - val_loss: 1.4875 - val_accuracy: 0.0000e+00
Ep

...

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
2/2 [==============================] - 3s 1s/step - loss: 0.7261 - accuracy: 0.6647 - val_loss: 1.4866 - val_accuracy: 0.0000e+00
Epoch 2/5
2/2 [==============================] - 1s 422ms/step - loss: 0.7213 - accuracy: 0.6679 - val_loss: 1.4872 - val_accuracy: 0.0000e+00
Epoch 3/5
2/2 [==============================] - 0s 196ms/step - loss: 0.7185 - accuracy: 0.6716 - val_loss: 1.4870 - val_accuracy: 0.0000e+00
Epoch 4/5
2/2 [==============================] - 0s 212ms/step - loss: 0.7164 - accuracy: 0.6726 - val_loss: 1.4864 - val_accuracy: 0.0000e+00
Epoch 5/5
2/2 [==============================] - 0s 180ms/step - loss: 0.7131 - accuracy: 0.6742 - val_loss: 1.4855 - val_accuracy: 0.0000e+00
Epoch 1/5
2/2 [==============================] - 2s 925ms/step - loss: 0.7287 - accuracy: 0.6511 - val_loss: 1.4829 - val_accuracy: 0.0000e+00
Epoch 2/5
2/2 [==============================] - 0s 195ms/step - loss: 0.7229 - accuracy: 0.6584 - val_loss: 1.4808 - val_accuracy: 0.0000e+00
Ep

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


119/119 [==============================] - 1s 9ms/step - loss: 1.2283 - accuracy: 0.5126


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.3115670680999756, 1.3251012563705444], [1.3063349723815918, 1.281938910484314], [1.2838425636291504, 1.2283215522766113]]
Accuracy for iterations:  [[0.8260276317596436, 0.5065858960151672], [0.7255564332008362, 0.5105373859405518], [0.774474561214447, 0.51264488697052]]
F1 for iterations:  [[0.8387103073098552, 0.4305368291447653], [0.789895075071453, 0.43480325825687666], [0.8195899266299259, 0.43727311539511304]]
Precision for iterations:  [[0.8710750247732294, 0.37747347291251343], [0.896647797206209, 0.38006896933018114], [0.9002973032216841, 0.3818529453842061]]
Recall for iterations:  [[0.8260276520553042, 0.5065858798735511], [0.7255564511129022, 0.5105374077976818], [0.774474548949098, 0.5126448893572181]]


### Cat5C

In [23]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Cat5C-Part5.csv')
test_cat = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Categories.csv')
test_cat_eq = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Categories_Eq.csv')

In [24]:

num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [25]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'IDCAT502.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=1024)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [26]:
def weiszfeld_update(points, tol=1e-5, max_iter=100):
    """
    Compute the geometric median using Weiszfeld's algorithm.

    Args:
    points: A list of points (numpy arrays) to find the geometric median of.
    tol: Tolerance for stopping criterion.
    max_iter: Maximum number of iterations.

    Returns:
    The geometric median of the points.
    """
    points = np.array(points)
    median = np.mean(points, axis=0)

    for _ in range(max_iter):
        distances = np.linalg.norm(points - median, axis=1)
        nonzero_distances = np.where(distances != 0, distances, np.finfo(float).eps)
        weights = 1 / nonzero_distances
        new_median = np.sum(points * weights[:, None], axis=0) / np.sum(weights)

        if np.linalg.norm(new_median - median) < tol:
            return new_median

        median = new_median

    return median

def aggregate(grad_list):
    """
    Apply Geometric Median aggregation method to a list of gradients.

    Args:
    grad_list: List of gradients from different models. Each element in the list
               is a list of gradients for each layer of a model.

    Returns:
    The geometric median of the gradients.
    """
    # Flatten gradients to compute geometric median
    flat_grads = [tf.concat([tf.reshape(g, [-1]) for g in grad], axis=0).numpy() for grad in grad_list]

    # Compute geometric median using Weiszfeld's algorithm
    flat_median = weiszfeld_update(flat_grads)

    # Reshape the flat median back to the original shape
    median_grad = []
    index = 0
    for grad in grad_list[0]:
        shape = tf.shape(grad)
        size = tf.reduce_prod(shape)
        median_grad.append(tf.reshape(flat_median[index:index + size], shape))
        index += size

    return median_grad

In [27]:
global_model = build_model((24,1,1))

In [28]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)
xc, yc = preprocessing(test_cat)
xce, yce = preprocessing(test_cat_eq)

In [29]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)

In [30]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3:
            x, y = x4, y4
        elif node == 4:
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xc, yc) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xce, yce)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idCAT502.hdf5')

Epoch 1/5
3/3 [==============================] - 2s 745ms/step - loss: 1.5416 - accuracy: 0.2265 - val_loss: 1.3802 - val_accuracy: 0.2298
Epoch 2/5
3/3 [==============================] - 0s 111ms/step - loss: 1.3504 - accuracy: 0.2600 - val_loss: 1.3722 - val_accuracy: 0.2899
Epoch 3/5
3/3 [==============================] - 0s 116ms/step - loss: 1.1727 - accuracy: 0.3531 - val_loss: 1.3639 - val_accuracy: 0.3568
Epoch 4/5
3/3 [==============================] - 0s 125ms/step - loss: 1.0143 - accuracy: 0.5560 - val_loss: 1.3553 - val_accuracy: 0.4545
Epoch 5/5
3/3 [==============================] - 0s 136ms/step - loss: 0.8717 - accuracy: 0.8584 - val_loss: 1.3463 - val_accuracy: 0.5540
Epoch 1/5
3/3 [==============================] - 2s 583ms/step - loss: 1.3488 - accuracy: 0.5225 - val_loss: 1.4506 - val_accuracy: 0.0000e+00
Epoch 2/5
3/3 [==============================] - 0s 157ms/step - loss: 1.2393 - accuracy: 0.5423 - val_loss: 1.4399 - val_accuracy: 0.0000e+00
Epoch 3/5
3/3 [====

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
3/3 [==============================] - 2s 730ms/step - loss: 2.2080 - accuracy: 0.6156 - val_loss: 1.9534 - val_accuracy: 0.0017
Epoch 2/5
3/3 [==============================] - 1s 168ms/step - loss: 2.0426 - accuracy: 0.6156 - val_loss: 1.8919 - val_accuracy: 0.0017
Epoch 3/5
3/3 [==============================] - 0s 127ms/step - loss: 1.8798 - accuracy: 0.6156 - val_loss: 1.8315 - val_accuracy: 0.0017
Epoch 4/5
3/3 [==============================] - 1s 198ms/step - loss: 1.7197 - accuracy: 0.6160 - val_loss: 1.7720 - val_accuracy: 0.0017
Epoch 5/5
3/3 [==============================] - 0s 137ms/step - loss: 1.5633 - accuracy: 0.6156 - val_loss: 1.7141 - val_accuracy: 0.0017
Epoch 1/5
3/3 [==============================] - 4s 1s/step - loss: 3.5534 - accuracy: 0.0000e+00 - val_loss: 1.7862 - val_accuracy: 0.0000e+00
Epoch 2/5
3/3 [==============================] - 0s 161ms/step - loss: 3.2775 - accuracy: 0.0000e+00 - val_loss: 1.7456 - val_accuracy: 0.0000e+00
Epoch 3/5
3/3 

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


119/119 [==============================] - 1s 5ms/step - loss: 1.4932 - accuracy: 0.4860


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
3/3 [==============================] - 5s 2s/step - loss: 2.4777 - accuracy: 0.6152 - val_loss: 3.3828 - val_accuracy: 0.0000e+00
Epoch 2/5
3/3 [==============================] - 1s 251ms/step - loss: 2.2792 - accuracy: 0.6152 - val_loss: 3.1503 - val_accuracy: 0.0000e+00
Epoch 3/5
3/3 [==============================] - 1s 167ms/step - loss: 2.0833 - accuracy: 0.6156 - val_loss: 2.9231 - val_accuracy: 0.0000e+00
Epoch 4/5
3/3 [==============================] - 1s 231ms/step - loss: 1.8889 - accuracy: 0.6156 - val_loss: 2.7015 - val_accuracy: 0.0000e+00
Epoch 5/5
3/3 [==============================] - 1s 206ms/step - loss: 1.6921 - accuracy: 0.6156 - val_loss: 2.4890 - val_accuracy: 0.0000e+00
Epoch 1/5
3/3 [==============================] - 4s 1s/step - loss: 3.7825 - accuracy: 0.0000e+00 - val_loss: 2.7244 - val_accuracy: 0.0000e+00
Epoch 2/5
3/3 [==============================] - 1s 235ms/step - loss: 3.4311 - accuracy: 0.0000e+00 - val_loss: 2.5684 - val_accuracy: 0.0000e+

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


119/119 [==============================] - 1s 6ms/step - loss: 2.0543 - accuracy: 0.4913


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.9910240173339844, 1.2652428150177002], [0.6150588989257812, 1.4932490587234497], [0.4834441840648651, 2.054264783859253]]
Accuracy for iterations:  [[0.8973122835159302, 0.48419389128685], [0.9244373440742493, 0.486037939786911], [0.9244373440742493, 0.4913066327571869]]
F1 for iterations:  [[0.8787738674644036, 0.3284296700570578], [0.9004414171259589, 0.33091816841135296], [0.9003289112623069, 0.33700442428759453]]
Precision for iterations:  [[0.8801373724548561, 0.2513008318600294], [0.8954919778941455, 0.2545545427585205], [0.8989584494355268, 0.2620911522597215]]
Recall for iterations:  [[0.8973122946245893, 0.48419388830347737], [0.9244373488746978, 0.48603793466807166], [0.9244373488746978, 0.49130663856691253]]
